In [0]:
%%time
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/pytorch/評価"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/pytorch/評価
CPU times: user 361 ms, sys: 135 ms, total: 496 ms
Wall time: 23.8 s


In [0]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.822s
     |████████████████████████████████| 686kB 2.8MB/s 
     |████████████████████████████████| 204kB 61.1MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=79177fadd371ed25c7cb1c11c4a57aba8355db1128fad48968fec9bc41b6a9fc
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=b779a482acd6387d21b3c6e9df7a20417ea575a533f3480a5775c8db2b1e02a2
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 114 ms, sys: 22.5 ms, total: 137 ms
Wall time: 16.6 s


In [0]:
%%time
import time
time.sleep(5)
! cat url.txt

In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [0]:
%%time
net = network.ResNet50(class_size=25)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
%%time
batch_size = 128
transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=True, Erasing=True)

train_dataset = torchvision.datasets.ImageFolder(root='./data/sign_mnist/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='./data/sign_mnist/test', transform=dl.val_transform())
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5), RandomRotation(degrees=(-180, 180), resample=False, expand=False), RandomPerspective(p=0.5), RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR), ToTensor(), <torchvision.transforms.transforms.RandomErasing object at 0x7f515e8e2e48>, Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]
CPU times: user 156 ms, sys: 15.4 ms, total: 172 ms
Wall time: 53.4 s


In [0]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 0 ns, sys: 39 ms, total: 39 ms
Wall time: 76.5 ms


In [0]:
%%time
model.train(data_loader, epoch_count=10)

cuda


Setting up a new session...


----------
Epoch 1/10 2020-01-25 15:22:58.085051
val Loss: 5.7030 Acc: 0.0343
----------
Epoch 2/10 2020-01-25 15:36:18.916936


In [0]:
%%time
model.predict(data_loader['val'])